In [ ]:
!nvidia-smi

Tue Mar 29 17:48:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
## mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 4.3 MB/s 
     |████████████████████████████████| 895 kB 59.5 MB/s 
     |████████████████████████████████| 596 kB 67.6 MB/s 
     |████████████████████████████████| 6.5 MB 61.4 MB/s 
     |████████████████████████████████| 67 kB 7.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
import pandas as pd
import os
pos_count = 0
neg_count = 0
for filename in os.listdir("./DeepLearningFinalProject/CodeContestsClean"):
  data_df = pd.read_csv("./DeepLearningFinalProject/CodeContestsClean/" + filename)
  data_df = data_df[data_df["Solution Language"] == "PYTHON3"]
  for index, row in data_df.iterrows():
    if pd.isnull(row["Incorrect Solution"]):
      pos_count += 1
    else:
      neg_count += 1

print("Positive Examples: " + str(pos_count))
print("Negative Examples: " + str(neg_count))

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (1,2,3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Positive Examples: 438996
Negative Examples: 611308


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils import data
from transformers import RobertaTokenizer, T5ForConditionalGeneration, AdamW, get_linear_schedule_with_warmup

class CodeContestDataset(data.Dataset):
    """CodeContest dataset."""

    def __init__(self, csv_file, tokenizer):
        """
        Args:
            csv_file (string): Path to the csv file
        """
        self.data = pd.read_csv(csv_file)
        self.data = self.data[self.data["Solution Language"] == "PYTHON3"]
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        label = None
        code = None
        if pd.isnull(self.data.iloc[idx, 5]):
          label = "1"
          code = self.data.iloc[idx, 4]
        else:
          label = "0"
          code = self.data.iloc[idx, 5]

        prefix_string = "Verify: " + str(self.data.iloc[idx, 2])
        ids_object = self.tokenizer(prefix_string + "</s>" + str(code), return_tensors="pt", padding='max_length')
        input_ids = ids_object.input_ids[0]
        attention_mask = ids_object.attention_mask[0]

        output_ids_object = self.tokenizer(label, return_tensors="pt")
        output_ids = output_ids_object.input_ids[0]
        output_attention_mask = output_ids_object.attention_mask[0]

        sample = {"source_ids": input_ids, "source_mask": attention_mask, "target_ids": output_ids, "target_mask": output_attention_mask}
        return sample

In [ ]:
all_datasets = []
tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-small')
for filename in os.listdir("./DeepLearningFinalProject/CodeContestsClean"):
  all_datasets.append(CodeContestDataset("./DeepLearningFinalProject/CodeContestsClean/" + filename, tokenizer))

dataset = data.ConcatDataset(all_datasets[:150])

Downloading:   0%|          | 0.00/687k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/287k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/12.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DtypeWarning: Columns (1,2,3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  after removing the cwd from sys.path.


In [ ]:
len(dataset)

781178

In [ ]:
for i in range(5):
  sample = dataset[i]
  print(tokenizer.decode(sample["source_ids"]))

<s>Verify: One day shooshuns found a sequence of n integers, written on a blackboard. The shooshuns can perform one operation with it, the operation consists of two steps:

  1. Find the number that goes k-th in the current sequence and add the same number to the end of the sequence; 
  2. Delete the first number of the current sequence. 



The shooshuns wonder after how many operations all numbers on the board will be the same and whether all numbers will ever be the same.

Input

The first line contains two space-separated integers n and k (1 ≤ k ≤ n ≤ 105).

The second line contains n space-separated integers: a1, a2,..., an (1 ≤ ai ≤ 105) — the sequence that the shooshuns found.

Output

Print the minimum number of operations, required for all numbers on the blackboard to become the same. If it is impossible to achieve, print -1.

Examples

Input

3 2
3 1 1


Output

1


Input

3 1
3 1 1


Output

-1

Note

In the first test case after the first operation the blackboard will have 

In [ ]:
!pip install pytorch_lightning

     |████████████████████████████████| 527 kB 4.2 MB/s 
     |████████████████████████████████| 134 kB 73.4 MB/s 
     |████████████████████████████████| 398 kB 93.7 MB/s 
     |████████████████████████████████| 829 kB 66.2 MB/s 
     |████████████████████████████████| 952 kB 61.0 MB/s 
     |████████████████████████████████| 1.1 MB 87.8 MB/s 
     |████████████████████████████████| 271 kB 80.2 MB/s 
     |████████████████████████████████| 144 kB 80.5 MB/s 
     |████████████████████████████████| 94 kB 4.5 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=497140bd3799ff8aae53aa32716b6527bf520b6471810e40a3e3ee2aa3b6a911
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: f

In [ ]:
import pytorch_lightning as pl
class T5FineTuner(pl.LightningModule):
  def __init__(self, hparams):
    super(T5FineTuner, self).__init__()
    self.hparams.update(vars(hparams))
    
    self.model = T5ForConditionalGeneration.from_pretrained(hparams.model_name_or_path)
    self.tokenizer = RobertaTokenizer.from_pretrained(hparams.tokenizer_name_or_path)
  
  def is_logger(self):
    return True
  
  def forward(
      self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, labels=None
  ):
    return self.model(
        input_ids,
        attention_mask=attention_mask,
        decoder_input_ids=decoder_input_ids,
        decoder_attention_mask=decoder_attention_mask,
        labels=labels,
    )

  def _step(self, batch):
    labels = batch["target_ids"]
    labels[labels[:, :] == self.tokenizer.pad_token_id] = -100

    outputs = self(
        input_ids=batch["source_ids"],
        attention_mask=batch["source_mask"],
        labels=labels,
        decoder_attention_mask=batch['target_mask']
    )

    loss = outputs[0]

    return loss

  def training_step(self, batch, batch_idx):
    loss = self._step(batch)

    tensorboard_logs = {"train_loss": loss}
    return {"loss": loss, "log": tensorboard_logs}
  
  def training_epoch_end(self, outputs):
    avg_train_loss = torch.stack([x["loss"] for x in outputs]).mean()
    #tensorboard_logs = {"avg_train_loss": avg_train_loss}
    self.log("avg_train_loss", avg_train_loss)

  def validation_step(self, batch, batch_idx):
    loss = self._step(batch)
    return {"val_loss": loss}
  
  def validation_epoch_end(self, outputs):
    avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
    tensorboard_logs = {"val_loss": avg_loss}
    return {"avg_val_loss": avg_loss, "log": tensorboard_logs, 'progress_bar': tensorboard_logs}

  def configure_optimizers(self):
    "Prepare optimizer and schedule (linear warmup and decay)"

    model = self.model
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": self.hparams.weight_decay,
        },
        {
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
        },
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=self.hparams.learning_rate, eps=self.hparams.adam_epsilon)
    self.opt = optimizer
    return [optimizer]
  
  def optimizer_step(self,
                     epoch=None, 
                    batch_idx=None, 
                    optimizer=None, 
                    optimizer_idx=None, 
                    optimizer_closure=None, 
                    on_tpu=None, 
                    using_native_amp=None, 
                    using_lbfgs=None
                     ):

    optimizer.step(closure=optimizer_closure)
    optimizer.zero_grad()
    self.lr_scheduler.step()
  
  def get_tqdm_dict(self):
    tqdm_dict = {"loss": "{:.3f}".format(self.trainer.avg_loss), "lr": self.lr_scheduler.get_last_lr()[-1]}

    return tqdm_dict

  def train_dataloader(self):
    dataloader = data.DataLoader(dataset, batch_size=self.hparams.train_batch_size, shuffle=True, num_workers=4)
    t_total = (
        (len(dataloader.dataset) // (self.hparams.train_batch_size * max(1, self.hparams.n_gpu)))
        // self.hparams.gradient_accumulation_steps
        * float(self.hparams.num_train_epochs)
    )
    scheduler = get_linear_schedule_with_warmup(
        self.opt, num_warmup_steps=self.hparams.warmup_steps, num_training_steps=t_total
    )
    self.lr_scheduler = scheduler
    return dataloader

  def val_dataloader(self):
    val_dataset = CodeContestDataset("./DeepLearningFinalProject/CodeContestsCleanValidationTesting/clean_validation.csv", tokenizer)
    test_dataset = CodeContestDataset("./DeepLearningFinalProject/CodeContestsCleanValidationTesting/clean_test.csv", tokenizer)
    val_test_dataset = data.ConcatDataset([val_dataset, test_dataset])
    return data.DataLoader(val_test_dataset, batch_size=self.hparams.eval_batch_size, num_workers=4)

In [ ]:
import logging
logger = logging.getLogger(__name__)

class LoggingCallback(pl.Callback):
  def on_validation_end(self, trainer, pl_module):
    logger.info("***** Validation results *****")
    if pl_module.is_logger():
      metrics = trainer.callback_metrics
      # Log results
      for key in sorted(metrics):
        if key not in ["log", "progress_bar"]:
          logger.info("{} = {}\n".format(key, str(metrics[key])))

  def on_test_end(self, trainer, pl_module):
    logger.info("***** Test results *****")

    if pl_module.is_logger():
      metrics = trainer.callback_metrics

      # Log and save results to file
      output_test_results_file = os.path.join(pl_module.hparams.output_dir, "test_results.txt")
      with open(output_test_results_file, "w") as writer:
        for key in sorted(metrics):
          if key not in ["log", "progress_bar"]:
            logger.info("{} = {}\n".format(key, str(metrics[key])))
            writer.write("{} = {}\n".format(key, str(metrics[key])))

In [ ]:
args_dict = dict(
    data_dir="",
    output_dir="./DeepLearningFinalProject",
    model_name_or_path='Salesforce/codet5-small',
    tokenizer_name_or_path='Salesforce/codet5-small',
    max_seq_length=512,
    learning_rate=1e-4,
    weight_decay=0.0,
    adam_epsilon=1e-8,
    warmup_steps=0,
    train_batch_size=16,
    eval_batch_size=16,
    num_train_epochs=1,
    gradient_accumulation_steps=8,
    n_gpu=1,
    fp_16=False,
    max_grad_norm=1.0,
    seed=42,
)

In [ ]:
import argparse
args = argparse.Namespace(**args_dict)

checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath=args.output_dir, monitor="val_loss", mode="min", save_top_k=5
)

train_params = dict(
    accumulate_grad_batches=args.gradient_accumulation_steps,
    gpus=args.n_gpu,
    max_epochs=args.num_train_epochs,
    precision= 16 if args.fp_16 else 32,
    gradient_clip_val=args.max_grad_norm,
    checkpoint_callback=checkpoint_callback,
    callbacks=[LoggingCallback()],
)

In [ ]:
model = T5FineTuner(args)

Downloading:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

In [ ]:
trainer = pl.Trainer(**train_params)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:148: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7f4161025750>)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7f4161025750>)`.
  f"Setting `Trainer(checkpoint_callback={checkpoint_callback})` is deprecated in v1.5 and will "
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [ ]:
trainer.fit(model)
torch.save(model.state_dict(), "./DeepLearningFinalProject/model_codet5_small_python3.pt")

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/configuration_validator.py:107: UserWarning: When using `Trainer(accumulate_grad_batches != 1)` and overriding `LightningModule.optimizer_{step,zero_grad}`, the hooks will not be called on every batch (rather, they are called on every optimization step).
  "When using `Trainer(accumulate_grad_batches != 1)` and overriding"
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 60.5 M
-----------------------------------------------------
60.5 M    Trainable params
0         Non-trainable para

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/optimization/closure.py:36: LightningDeprecationWarning: One of the returned values {'log'} has a `grad_fn`. We will detach it automatically but this behaviour will change in v1.6. Please detach it manually: `return {'loss': ..., 'something': something.detach()}`
  f"One of the returned values {set(extra.keys())} has a `grad_fn`. We will detach it automatically"


Validating: 0it [00:00, ?it/s]

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
from sklearn import metrics
model.model.eval()
model.model.to(device)
test_dataset = data.ConcatDataset(all_datasets[150:])
loader = data.DataLoader(test_dataset, batch_size=16, num_workers=4)
actual = []
predicted = []
for batch_idx, batch in enumerate(loader):
  outs = model.model.generate(input_ids=batch['source_ids'].to(device), 
                              attention_mask=batch['source_mask'].to(device), 
                              max_length=4)
  dec = [tokenizer.decode(ids, skip_special_tokens=True) for ids in outs]
  targets = [tokenizer.decode(ids, skip_special_tokens=True) for ids in batch['target_ids']]
  for i in range(len(targets)):

    actual.extend(targets[i])
    predicted.extend(dec[i])

metrics.accuracy_score(actual, predicted)

0.8039245557842795

In [ ]:
print(metrics.classification_report(actual, predicted))

              precision    recall  f1-score   support

           0       0.82      0.85      0.84    157291
           1       0.78      0.73      0.76    111835

    accuracy                           0.80    269126
   macro avg       0.80      0.79      0.80    269126
weighted avg       0.80      0.80      0.80    269126

